In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import cycle

import os
for dirname, _, filenames in os.walk('./data_sets/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

In [ ]:
INPUT_DIR_PATH = './data_sets/'

sell_prices_df = pd.read_csv(INPUT_DIR_PATH + 'sell_prices.csv')
calendar_df = pd.read_csv(INPUT_DIR_PATH + 'calendar.csv')
sales_train_validation_df = pd.read_csv(INPUT_DIR_PATH + 'test/sales_train_validation.csv')
submission_df = pd.read_csv(INPUT_DIR_PATH + 'sample_submission.csv')

In [ ]:
print(f'Shape of sell_prices_df is: {sell_prices_df.shape}')
print(f'Shape of calendar_df is: {calendar_df.shape}')
print(f'Shape of sales_train_validation_df is: {sales_train_validation_df.shape}')
print(f'Shape of submission_df is: {submission_df.shape}')

In [ ]:
sell_prices_df.head()

In [ ]:
calendar_df.head()

In [ ]:
sales_train_validation_df.head()

In [ ]:
submission_df.head()

In [ ]:
# selecting 10 random rows from dataframe
stvd10 = sales_train_validation_df.sample(n = 10)
d_cols = [c for c in stvd10.columns if 'd_' in c] # sales data columns
stvd10 = stvd10.set_index('id')[d_cols].T

In [ ]:
plt.figure(figsize=(40, 40))
plt.subplots_adjust(top=1.2, hspace = 0.8)
for i,item_id in enumerate(list(stvd10.columns)):
    plt.subplot(5, 2, i + 1)
    stvd10[item_id].plot(figsize=(20, 12),
          title=f'{item_id} sales by "d" number',
          color=next(color_cycle))
    plt.grid(False)


In [ ]:
cal = calendar_df[['d','date','event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI']]

In [ ]:
stvd_1 = sales_train_validation_df.set_index('id')[d_cols].T
# rename id column to 'd', to perform merge operation
stvd_1 = stvd_1.reset_index().rename(columns={'index': 'd'})
# merging df cal and sales_train_validation_df on 'd'
stvd_merged = stvd_1.merge(cal, how='left', validate='1:1')

In [ ]:
stvd10.head()

In [ ]:
# rename id column to 'd', to perform merge operation
stvd10 = stvd10.reset_index().rename(columns={'index': 'd'})
stvd10 = stvd10.merge(cal, how='left', validate='1:1')
stvd10_date = stvd10.set_index('date')

In [ ]:
stvd10_date.head()

In [ ]:
plt.figure(figsize=(40, 40))
plt.subplots_adjust(top=1.2, hspace = 0.8)
for i,item_id in enumerate(list(stvd10_date.columns[1:11])):
    plt.subplot(5, 2, i + 1)
    stvd10_date[item_id].plot(figsize=(20, 12),
          title=f'{item_id} sales by "d" number',
          color=next(color_cycle))
    plt.tight_layout()
    plt.grid(False)

In [27]:
last_thirty_day_avg_sales = sales_train_validation_df.set_index('id')[d_cols[-30:]].mean(axis=1).to_dict()
fcols = [f for f in submission_df.columns if 'F' in f]
for f in fcols:
    submission_df[f] = submission_df['id'].map(last_thirty_day_avg_sales).fillna(0)
    
submission_df.to_csv('submission.csv', index=False)

In [ ]:
submission_df.head()